In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
archivos = {
    2022: "Nacional EPEN Trim. Abr-May-Jun 2022.csv",
    2023: "Trim_Abr_May_Jun23.csv",
    2024: "Trim_Abr_May_Jun24.csv",
    2025: "Trim_Abr_May_Jun25.csv",
}

In [ ]:
# Asegurar tipos numéricos clave
ING_MIN_REF_POR_ANIO = {
    2022: 1025,  
    2023: 1025,
    2024: 1025,
    2025: 1130,
}

def tasa_pond(df, var, w):
    if var not in df.columns or w not in df.columns:
        return np.nan
    d = df[[var, w]].dropna()
    if len(d) == 0 or d[w].sum() == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

def prom_pond(df, var, w):
    if var not in df.columns or w not in df.columns:
        return np.nan
    d = df[[var, w]].dropna()
    if len(d) == 0 or d[w].sum() == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

def obtener_peso(df):
    if "FAC_T300" in df.columns:
        return "FAC_T300"
    fa_cols = [c for c in df.columns if str(c).lower().startswith("fa_")]
    if fa_cols:
        return fa_cols[0]
    raise ValueError("No encontré factor de expansión (FAC_T300 o fa_*).")

def obtener_col_deseo(df):
    if "C333" in df.columns:
        return "C333"
    if "P209H" in df.columns:
        return "P209H"
    return None

def preparar_df(df, anio):
    df = df.copy()

    peso = obtener_peso(df)
    df[peso] = pd.to_numeric(df[peso], errors="coerce")

    # tipos
    for c in ["C208", "INGTOTP", "whoraT", "OCUP300", "SEGURO1"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # grupo etario
    df["grupo_edad"] = np.select(
        [df["C208"].between(18, 29), df["C208"].between(30, 49)],
        ["Jóvenes (18–29)", "Adultos (30–49)"],
        default=None
    )
    df = df[df["grupo_edad"].notna()].copy()

    # ocupado
    df["ocupado"] = np.where(df["OCUP300"] == 1, 1, 0)

    # desea más horas de trabajo
    col_deseo = obtener_col_deseo(df)
    if col_deseo is not None:
        df[col_deseo] = pd.to_numeric(df[col_deseo], errors="coerce")
        df["quiere_mas_horas"] = np.where(df[col_deseo] == 1, 1,
                                  np.where(df[col_deseo] == 2, 0, np.nan))
    else:
        df["quiere_mas_horas"] = np.nan

    # seguro de salud
    if "SEGURO1" in df.columns:
        df["con_seguro_salud"] = np.where(df["SEGURO1"] == 6, 0,
                                  np.where(df["SEGURO1"].isna(), np.nan, 1))
    else:
        df["con_seguro_salud"] = np.nan

    # empleo adecuado
    ing_min = ING_MIN_REF_POR_ANIO.get(anio, 1130)
    cond_fulltime_bien_pagado = (df["whoraT"] >= 35) & (df["INGTOTP"] >= ing_min)
    cond_parttime_sin_deseo   = (df["whoraT"] < 35) & (df["quiere_mas_horas"] == 0)

    df["empleo_adecuado"] = np.where(
        (df["ocupado"] == 1) & (cond_fulltime_bien_pagado | cond_parttime_sin_deseo),
        1,
        np.where(df["ocupado"] == 1, 0, np.nan)
    )

    return df, peso

In [ ]:
resultados = []

for anio, archivo in archivos.items():
    df_raw = pd.read_csv(archivo)
    df_proc, peso = preparar_df(df_raw, anio)

    for grupo in ["Jóvenes (18–29)", "Adultos (30–49)"]:
        dfg_oc = df_proc[(df_proc["grupo_edad"] == grupo) & (df_proc["ocupado"] == 1)]

        resultados.append({
            "anio": anio,
            "grupo": grupo,
            "empleo_adecuado": tasa_pond(dfg_oc, "empleo_adecuado", peso) * 100,
            "con_seguro_salud": tasa_pond(dfg_oc, "con_seguro_salud", peso) * 100,
            "ingreso_prom": prom_pond(dfg_oc, "INGTOTP", peso),
            "horas_prom": prom_pond(dfg_oc, "whoraT", peso),
        })

df_series = pd.DataFrame(resultados).sort_values(["grupo", "anio"])
df_series

In [ ]:
def tasa_pond(df, var, w):
    d = df[[var, w]].dropna()
    if len(d) == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

def prom_pond(df, var, w):
    d = df[[var, w]].dropna()
    if len(d) == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

In [ ]:
def resumen_grupo(df, grupo):
    dfg = df[df["grupo_edad"] == grupo]
    dfg_oc = dfg[dfg["ocupado"] == 1]

    return pd.Series({
        "Tasa de ocupación": tasa_pond(dfg, "ocupado", "fa_amj25"),
        "Empleo adecuado (ocupados)": tasa_pond(dfg_oc, "empleo_adecuado", "fa_amj25"),
        "Con seguro de salud (ocupados)": tasa_pond(dfg_oc, "con_seguro_salud", "fa_amj25"),
        "Ingreso promedio (ocupados)": prom_pond(dfg_oc, "INGTOTP", "fa_amj25")
    })

In [ ]:
resultados = []

for anio, archivo in archivos.items():
    df_raw = pd.read_csv(archivo)
    df_proc, peso = preparar_df(df_raw, anio)

    for grupo in ["Jóvenes (18–29)", "Adultos (30–49)"]:
        dfg_oc = df_proc[(df_proc["grupo_edad"] == grupo) & (df_proc["ocupado"] == 1)]

        resultados.append({
            "anio": anio,
            "grupo": grupo,
            "empleo_adecuado": tasa_pond(dfg_oc, "empleo_adecuado", peso) * 100,
            "con_seguro_salud": tasa_pond(dfg_oc, "con_seguro_salud", peso) * 100,
            "ingreso_prom": prom_pond(dfg_oc, "INGTOTP", peso),
            "horas_prom": prom_pond(dfg_oc, "whoraT", peso),
        })

df_series = pd.DataFrame(resultados).sort_values(["grupo", "anio"])
df_series

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))

for grupo, dfg in df_series.groupby("grupo"):
    ax.plot(dfg["anio"], dfg["empleo_adecuado"], marker="o", linewidth=2, label=grupo)

ax.set_title("Adequate employment by group age\nApr-May-Jun Quarter (2022–2025) - EPEN INEI")
ax.set_ylabel("Percentage (%)")
ax.set_xlabel("Año")
ax.set_ylim(0, 105)
ax.grid(alpha=0.3)
ax.legend(title="Age group")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ING_MIN_REF = 1130  # proxy RMV documentado

def tasa_pond(df, var, w):
    # tasa ponderada para variables binarias 0/1 (o proporciones)
    if var not in df.columns or w not in df.columns:
        return np.nan
    d = df[[var, w]].dropna()
    if len(d) == 0:
        return np.nan
    if d[w].sum() == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

def prom_pond(df, var, w):
    # promedio ponderado para variables continuas
    if var not in df.columns or w not in df.columns:
        return np.nan
    d = df[[var, w]].dropna()
    if len(d) == 0:
        return np.nan
    if d[w].sum() == 0:
        return np.nan
    return (d[var] * d[w]).sum() / d[w].sum()

def obtener_peso(df):
    # 2022 puede traer FAC_T300 
    if "FAC_T300" in df.columns:
        return "FAC_T300"
    # caso estándar: fa_*
    fa_cols = [c for c in df.columns if str(c).lower().startswith("fa_")]
    if fa_cols:
        return fa_cols[0]
    raise ValueError("No encontré factor de expansión (FAC_T300 o fa_*).")

def obtener_col_deseo_mas_horas(df):
    # 2024-2025: C333 | 2022-2023: P209H 
    if "C333" in df.columns:
        return "C333"
    if "P209H" in df.columns:
        return "P209H"
    return None

def preparar_df_inei(df, ing_min_ref=ING_MIN_REF):
    df = df.copy()

    # peso
    peso = obtener_peso(df)
    df[peso] = pd.to_numeric(df[peso], errors="coerce")

    # tipos base
    for c in ["C208", "INGTOTP", "whoraT", "OCUP300"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # grupo etario
    df["grupo_edad"] = np.select(
        [(df["C208"].between(18, 29)), (df["C208"].between(30, 49))],
        ["Jóvenes (18–29)", "Adultos (30–49)"],
        default=None
    )
    df = df[df["grupo_edad"].notna()].copy()

    # ocupado
    df["ocupado"] = np.where(df["OCUP300"] == 1, 1, 0)

    # desea más horas de trabajo
    col_deseo = obtener_col_deseo_mas_horas(df)
    if col_deseo is None:
        df["quiere_mas_horas"] = np.nan
    else:
        df[col_deseo] = pd.to_numeric(df[col_deseo], errors="coerce")
        df["quiere_mas_horas"] = np.where(df[col_deseo] == 1, 1,
                                  np.where(df[col_deseo] == 2, 0, np.nan))

    # seguro de salud (robusto)
    # si no existe SEGURO1 en un archivo, dejamos NaN para no romper
    if "SEGURO1" in df.columns:
        df["SEGURO1"] = pd.to_numeric(df["SEGURO1"], errors="coerce")
        df["con_seguro_salud"] = np.where(
            df["SEGURO1"] == 6, 0,
            np.where(df["SEGURO1"].isna(), np.nan, 1)
        )
    else:
        df["con_seguro_salud"] = np.nan

    # empleo adecuado (INEI) -> solo definido para ocupados
    cond_fulltime_bien_pagado = (df["whoraT"] >= 35) & (df["INGTOTP"] >= ing_min_ref)
    cond_parttime_sin_deseo   = (df["whoraT"] < 35) & (df["quiere_mas_horas"] == 0)

    df["empleo_adecuado"] = np.where(
        (df["ocupado"] == 1) & (cond_fulltime_bien_pagado | cond_parttime_sin_deseo),
        1,
        np.where(df["ocupado"] == 1, 0, np.nan)
    )

    return df, peso

def resumen_por_anio(df, peso, anio):
    out = []
    for grupo in ["Jóvenes (18–29)", "Adultos (30–49)"]:
        dfg_oc = df[(df["grupo_edad"] == grupo) & (df["ocupado"] == 1)]

        out.append({
            "anio": anio,
            "grupo": grupo,
            "empleo_adecuado": tasa_pond(dfg_oc, "empleo_adecuado", peso) * 100,
            "con_seguro_salud": tasa_pond(dfg_oc, "con_seguro_salud", peso) * 100,
            "ingreso_prom": prom_pond(dfg_oc, "INGTOTP", peso)
        })
    return pd.DataFrame(out)

In [ ]:
df25 = pd.read_csv("Trim_Abr_May_Jun25.csv")
df25["C333"].value_counts(dropna=False).head(10)

In [ ]:
df25_proc, w25 = preparar_df_inei(df25)
df25_proc[df25_proc["ocupado"]==1]["empleo_adecuado"].value_counts(dropna=False)

In [ ]:
df25 = pd.read_csv("Trim_Abr_May_Jun25.csv")
df25_proc, w = preparar_df_inei(df25)
df25_oc = df25_proc[df25_proc["ocupado"] == 1].copy()

df25_oc[["whoraT", "INGTOTP"]].describe(percentiles=[.05,.1,.25,.5,.75,.9,.95])

In [ ]:
df25_oc["whoraT"].value_counts().head(15)
df25_oc["INGTOTP"].value_counts().head(15)

In [ ]:
df25_oc[["whoraT", "INGTOTP"]].describe(percentiles=[.05,.1,.25,.5,.75,.9,.95])

In [ ]:
def resumen_ingreso(df, peso, anio):
    out = []
    for grupo in ["Jóvenes (18–29)", "Adultos (30–49)"]:
        dfg = df[(df["grupo_edad"] == grupo) & (df["ocupado"] == 1)]
        out.append({
            "anio": anio,
            "grupo": grupo,
            "ingreso_prom": prom_pond(dfg, "INGTOTP", peso)
        })
    return pd.DataFrame(out)

In [ ]:
ingresos = []
for anio, archivo in archivos.items():
    df_raw = pd.read_csv(archivo)
    df_proc, peso = preparar_df_inei(df_raw)
    ingresos.append(resumen_ingreso(df_proc, peso, anio))

df_ing = pd.concat(ingresos, ignore_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for grupo, dfg in df_ing.groupby("grupo"):
    ax.plot(
        dfg["anio"],
        dfg["ingreso_prom"],
        marker="o",
        linewidth=2,
        label=grupo
    )

ax.set_title("Average monthly income by age group \nEPEN Apr-May-Jun Quarter (2022–2025) - EPEN INEI")
ax.set_ylabel("Soles (PEN)")
ax.set_xlabel("Year")
ax.grid(alpha=0.3)
ax.legend(title="Age group")

plt.tight_layout()
plt.show()

In [ ]:
def resumen_horas(df, peso, anio):
    out = []
    for grupo in ["Jóvenes (18–29)", "Adultos (30–49)"]:
        dfg = df[(df["grupo_edad"] == grupo) & (df["ocupado"] == 1)]
        out.append({
            "anio": anio,
            "grupo": grupo,
            "horas_prom": prom_pond(dfg, "whoraT", peso)
        })
    return pd.DataFrame(out)

In [ ]:
horas = []
for anio, archivo in archivos.items():
    df_raw = pd.read_csv(archivo)
    df_proc, peso = preparar_df_inei(df_raw)
    horas.append(resumen_horas(df_proc, peso, anio))

df_horas = pd.concat(horas, ignore_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for grupo, dfg in df_horas.groupby("grupo"):
    ax.plot(
        dfg["anio"],
        dfg["horas_prom"],
        marker="o",
        linewidth=2,
        label=grupo
    )

ax.axhline(35, linestyle="--", color="gray", alpha=0.6, label="35 hours Threshold")
ax.set_title("Average work hours by group age \nEPEN Apr-May-Jun Quarter (2022–2025) - EPEN INEI")
ax.set_ylabel("Weekly work hours")
ax.set_xlabel("Year")
ax.grid(alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
resultados = []
for anio, archivo in archivos.items():
    df_raw = pd.read_csv(archivo)
    df_proc, peso = preparar_df_inei(df_raw)
    resultados.append(resumen_por_anio(df_proc, peso, anio))

df_series = pd.concat(resultados, ignore_index=True).sort_values(["grupo", "anio"])
df_series

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

for grupo, dfg in df_series.groupby("grupo"):
    ax.plot(dfg["anio"], dfg["con_seguro_salud"], marker="o", linewidth=2, label=grupo)

ax.set_title("Access to health insurance by group age\nEPEN Apr-May-Jun Quarter  (2023–2025) - EPEN INEI")
ax.set_ylabel("Percentage (%)")
ax.set_xlabel("Year")
ax.set_ylim(0, 100)
ax.grid(alpha=0.3)
ax.legend(title="Age group")

plt.tight_layout()
plt.show()